## Code for LLaMa-LORA Tuning

Find the code for tuning the LLaMa-7B mode using the LORA method below. It is an implementation of LLaMA-tuninig which is "fully open source under the Apache 2.0 license."

https://github.com/Lightning-AI/lit-llama/blob/main/howto/finetune_adapter.md

In [ ]:
!pip install huggingface
!pip install huggingface_hub


     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 268.8/268.8 kB 21.2 MB/s eta 0:00:00


In [ ]:
!huggingface-cli login



    _|    _|  _|    _|    _|_|_|    _|_|_|  _|_|_|  _|      _|    _|_|_|      _|_|_|_|    _|_|      _|_|_|  _|_|_|_|
    _|    _|  _|    _|  _|        _|          _|    _|_|    _|  _|            _|        _|    _|  _|        _|
    _|_|_|_|  _|    _|  _|  _|_|  _|  _|_|    _|    _|  _|  _|  _|  _|_|      _|_|_|    _|_|_|_|  _|        _|_|_|
    _|    _|  _|    _|  _|    _|  _|    _|    _|    _|    _|_|  _|    _|      _|        _|    _|  _|        _|
    _|    _|    _|_|      _|_|_|    _|_|_|  _|_|_|  _|      _|    _|_|_|      _|        _|    _|    _|_|_|  _|_|_|_|
    
    To login, `huggingface_hub` requires a token generated from https://huggingface.co/settings/tokens .
Token: 
Add token as git credential? (Y/n) Y
Token is valid (permission: read).
Cannot authenticate through git-credential as no helper is defined on your machine.
You might have to re-authenticate when pushing to the Hugging Face Hub.
Run the following command in your terminal in case you want to set the 'store' cre

In [ ]:
from huggingface_hub import notebook_login

notebook_login()

https://github.com/Lightning-AI/lit-llama/blob/main/howto/finetune_adapter.md

In [ ]:
!git clone https://github.com/Lightning-AI/lit-llama ## this first ?!

Cloning into 'lit-llama'...
remote: Enumerating objects: 1845, done.
remote: Counting objects: 100% (230/230), done.
remote: Compressing objects: 100% (135/135), done.
remote: Total 1845 (delta 134), reused 167 (delta 95), pack-reused 1615
Receiving objects: 100% (1845/1845), 1.65 MiB | 2.32 MiB/s, done.
Resolving deltas: 100% (1133/1133), done.


In [ ]:
!git clone https://huggingface.co/meta-llama/Llama-2-7b-hf checkpoints/open-llama/7B



Cloning into 'checkpoints/open-llama/7B'...
remote: Enumerating objects: 27, done.
remote: Counting objects: 100% (11/11), done.
remote: Compressing objects: 100% (11/11), done.
remote: Total 27 (delta 3), reused 0 (delta 0), pack-reused 16
Unpacking objects: 100% (27/27), 495.01 KiB | 1.02 MiB/s, done.
Filtering content: 100% (7/7), 9.65 GiB | 26.82 MiB/s, done.
Encountered 4 file(s) that may not have been copied correctly on Windows:
	pytorch_model-00003-of-00003.bin
	pytorch_model-00001-of-00003.bin
	model-00001-of-00002.safetensors
	pytorch_model-00002-of-00003.bin

See: `git lfs help smudge` for more details.


In [ ]:
!pip install -r lit-llama/requirements.txt
# might take some time dpeneding on internet connectivity


  Cloning https://github.com/Lightning-AI/lightning (to revision master) to /tmp/pip-install-_2n1hi8d/lightning_752d0be65c854596a5db2c7349c80f71
  Running command git clone --filter=blob:none --quiet https://github.com/Lightning-AI/lightning /tmp/pip-install-_2n1hi8d/lightning_752d0be65c854596a5db2c7349c80f71
  Resolved https://github.com/Lightning-AI/lightning to commit dab373de548b352a69c8ea7251937fb4663038dc
  Running command git submodule update --init --recursive -q
  Encountered 31 file(s) that should have been pointers, but weren't:
        .notebooks/course_UvA-DL/01-introduction-to-pytorch.ipynb
        .notebooks/course_UvA-DL/02-activation-functions.ipynb
        .notebooks/course_UvA-DL/03-initialization-and-optimization.ipynb
        .notebooks/course_UvA-DL/04-inception-resnet-densenet.ipynb
        .notebooks/course_UvA-DL/05-transformers-and-MH-attention.ipynb
        .notebooks/course_UvA-DL/06-graph-neural-networks.ipynb
        .notebooks/course_UvA-DL/07-deep-energy

In [ ]:
!python lit-llama/scripts/convert_hf_checkpoint.py --checkpoint_dir checkpoints/open-llama/7B --model_size 7B

Initializing lit-llama
Saving to disk at checkpoints/lit-llama/7B
Processing checkpoints/open-llama/7B/pytorch_model-00001-of-00003.bin
Processing checkpoints/open-llama/7B/pytorch_model-00002-of-00003.bin
Processing checkpoints/open-llama/7B/pytorch_model-00003-of-00003.bin


## Finetuning try 1

https://github.com/Lightning-AI/lit-llama/blob/main/howto/finetune_adapter.md

In [ ]:
# connecting to drive
from google.colab import drive
drive.mount('/content/gdrive')


Mounted at /content/gdrive


### Loading in data from drive and setting up own dataloader (.pt) files like the function in the python file prepare_alpacy do.

In [ ]:

import pandas as pd
airbnb_london_filtered_images = pd.read_csv("/content/gdrive/My Drive/Thesis/London_Data/discriminator_tabular_data_images.csv")

In [ ]:
import numpy as np
frac_train_size = 0.7

train_eval_airbnb_london_filtered_images = airbnb_london_filtered_images[airbnb_london_filtered_images.in_top_third == 1]

train_airbnb_london_filtered_advanced = train_eval_airbnb_london_filtered_images.sample(n = int(np.ceil(frac_train_size*train_eval_airbnb_london_filtered_images.shape[0])), random_state = 100)
eval_airbnb_london_filtered_advanced = train_eval_airbnb_london_filtered_images.drop(train_airbnb_london_filtered_advanced.index, axis = 0)

train_airbnb_london_filtered_advanced.index = list(range(train_airbnb_london_filtered_advanced.shape[0]))
eval_airbnb_london_filtered_advanced.index = list(range(eval_airbnb_london_filtered_advanced.shape[0]))


Logical dependencies of functions in prepare_alpaca file

'prepare' uses
> 'prepare_sample' uses
>> 'tokenzize' & 'generate promt'



In [ ]:
## Using the following instruction:

instruction = "Summarize the following description into a short title for an AirBnB listing."


In [ ]:
import os
from pathlib import Path
from typing import Optional
import torch
from sentencepiece import SentencePieceProcessor, SentencePieceTrainer
import sys
import torch
import requests
import json
from torch.utils.data import random_split
from tqdm import tqdm

class Tokenizer:
    """Tokenizer for LLaMA."""

    def __init__(self, model_path: Path) -> None:
        self.processor = SentencePieceProcessor(model_file=str(model_path))
        self.bos_id = self.processor.bos_id()
        self.eos_id = self.processor.eos_id()
        self.pad_id = self.processor.pad_id()

    @property
    def vocab_size(self) -> int:
        return self.processor.vocab_size()

    def encode(
        self,
        string: str,
        bos: bool = True,
        eos: bool = False,
        max_length: int = -1,
        pad: bool = False,
        device: Optional[torch.device] = None
    ) -> torch.Tensor:
        tokens = self.processor.encode(string)
        if bos:
            tokens = [self.bos_id] + tokens
        if eos:
            tokens = tokens + [self.eos_id]
        if max_length > 0:
            tokens = tokens[:max_length]
        if pad and len(tokens) < max_length:
            tokens += [self.pad_id] * (max_length - len(tokens))

        return torch.tensor(tokens, dtype=torch.int, device=device)

    def decode(self, tokens: torch.Tensor) -> str:
        return self.processor.decode(tokens.tolist())

    @staticmethod
    def train(input: str, destination: str, vocab_size=32000) -> None:
        model_prefix = os.path.join(destination, "tokenizer")
        SentencePieceTrainer.Train(input=input, model_prefix=model_prefix, vocab_size=vocab_size)


## general function
def tokenize(tokenizer: Tokenizer, string: str, max_length: int, eos=True) -> torch.Tensor:
    return tokenizer.encode(string, bos=True, eos=eos, max_length=max_length)

## general function

def generate_prompt(example):
    """Generates a standardized message to prompt the model with an instruction, optional input and a
    'response' field."""

    if example["input"]:
        return (
            "Below is an instruction that describes a task, paired with an input that provides further context. "
            "Write a response that appropriately completes the request.\n\n"
            f"### Instruction:\n{example['instruction']}\n\n### Input:\n{example['input']}\n\n### Response:"
        )
    return (
        "Below is an instruction that describes a task. "
        "Write a response that appropriately completes the request.\n\n"
        f"### Instruction:\n{example['instruction']}\n\n### Response:"
    )


def prepare_sample(example: dict, tokenizer: Tokenizer, max_length: int, mask_inputs: bool = True):
    """Processes a single sample.

    Each sample in the dataset consists of:
    - instruction: A string describing the task
    - input: A string holding a special input value for the instruction.
        This only applies to some samples, and in others this is empty.
    - output: The response string

    This function processes this data to produce a prompt text and a label for
    supervised training. The input text is formed as a single message including all
    the instruction, the input (optional) and the response.
    The label/target is the same message but can optionally have the instruction + input text
    masked out (mask_inputs=True).

    Finally, both the prompt and the label get tokenized. If desired, all tokens
    in the label that correspond to the original input prompt get masked out (default).
    """

    full_prompt = generate_prompt(example)
    full_prompt_and_response = full_prompt + example["output"]
    encoded_full_prompt = tokenize(tokenizer, full_prompt, max_length=max_length, eos=False)
    encoded_full_prompt_and_response = tokenize(tokenizer, full_prompt_and_response, eos=True, max_length=max_length)

    # The labels are the full prompt with response, but with the prompt masked out
    labels = encoded_full_prompt_and_response.clone()
    if mask_inputs:
        labels[:len(encoded_full_prompt)]  = -1 ## corresponds to "ignore index" inprepare_alpaca.py file


    return {**example, "input_ids": encoded_full_prompt_and_response, "input_ids_no_response": encoded_full_prompt, "labels": labels}




#### prepare function is not required!

In [ ]:
tokenizer = Tokenizer("/content/checkpoints/lit-llama/tokenizer.model")

Reocding the prepare function:

In [ ]:
## setting up new dict

data_dict_train  = []
data_dict_eval  = []

In [ ]:

for i in range(train_airbnb_london_filtered_advanced.shape[0]):

  data_dict_train.append({
      "instruction": "Summarize the following description into a short title for an AirBnB listing.",
      "input":   train_airbnb_london_filtered_advanced.description[i],
      "output": train_airbnb_london_filtered_advanced.name[i]
  })

for i in range(eval_airbnb_london_filtered_advanced.shape[0]):

  data_dict_eval.append({
      "instruction": "Summarize the following description into a short title for an AirBnB listing.",
      "input":   eval_airbnb_london_filtered_advanced.description[i],
      "output": eval_airbnb_london_filtered_advanced.name[i]
  })


Create a new folder called "data"

In [ ]:
!mkdir data

In [ ]:
print("Processing train split ...")
max_seq_length = 256
mask_inputs = True
train_set = [prepare_sample(sample, tokenizer, max_seq_length, mask_inputs) for sample in data_dict_train]
torch.save(train_set, "/content/data/train.pt")




Processing train split ...


In [ ]:
print("Processing test split ...")
max_seq_length = 256
mask_inputs = True
test_set = [prepare_sample(sample, tokenizer, max_seq_length, mask_inputs) for sample in data_dict_eval]
torch.save(train_set, "/content/data/test.pt")


Processing test split ...


In [ ]:
epoch_size = len(train_set)
epoch_size


4048

#### Actual fine-tuning

The finetuning file is called. However, a few changes need to be made to this file (lora.py)

* in line 39, the size must be set to 4048 (size of my training set)
* in line 42, the lora_r hyper-param can be adjusted. I kept it at r = 8 as was suggested by LORA authors!
* in line 49, 'data_dict' must be set to "data"
* in line 52, 'out_dir' can be set to "out/adapter/airbnb" (Not necessary, just nicer!)

In [ ]:
!python lit-llama/finetune/lora.py

/content/lit-llama/finetune/lora.py:216: JsonargparseDeprecationWarning: 
    Only use the public API as described in https://jsonargparse.readthedocs.io/en/stable/#api-reference.
    Importing from jsonargparse.cli is kept only to avoid breaking code that does not correctly use the public
    API. It will no longer be available from v5.0.0.

  from jsonargparse.cli import CLI
Global seed set to 1337
iter 0: loss nan, time: 2135.83ms
iter 1: loss 3.1275, time: 246.10ms
iter 2: loss 3.4024, time: 244.65ms
iter 3: loss nan, time: 242.48ms
iter 4: loss nan, time: 240.86ms
iter 5: loss 2.5819, time: 240.69ms
iter 6: loss 2.5325, time: 241.28ms
iter 7: loss 5.5841, time: 241.41ms
iter 8: loss 4.7928, time: 240.40ms
iter 9: loss 4.5195, time: 240.07ms
iter 10: loss 4.7221, time: 240.29ms
iter 11: loss 3.0495, time: 240.61ms
iter 12: loss 3.3865, time: 240.51ms
iter 13: loss 3.3496, time: 241.29ms
iter 14: loss nan, time: 240.97ms
iter 15: loss 3.9006, time: 240.90ms
iter 16: loss 3.2090, tim

## Evaluating and generating titles after the Fine-Tuning

Firstly: computation of ROUGE Scores

In [ ]:
import nltk
nltk.download('punkt')

def postprocess_text(text_sequence):   ##, labels):

    """
    Post-processing to prepare inputs to the ROGUE functions
    """

    text_sequence = [a.strip() for a in text_sequence]

    # ROUGE expects a newline after each sentence
    text_sequence = ["\n".join(nltk.sent_tokenize(a)) for a in text_sequence]

    return text_sequence

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.


In [ ]:
!mkdir out

In [ ]:
#import torch

model_ft = torch.load("/content/gdrive/My Drive/Thesis/Models/lit-llama-lora-finetuned_v2.pth")
torch.save(model_ft, "/content/out/lit-llama-adapter-finetuned.pth")


In [ ]:
sys.path.append("/content/lit-llama")


import lightning as L
import torch
from generate import generate
from lit_llama import Tokenizer
from lit_llama.adapter import LLaMA
from lit_llama.utils import EmptyInitOnDevice, lazy_load, llama_model_lookup
from scripts.prepare_alpaca import generate_prompt

In [ ]:
adapter_path ="/content/out/lit-llama-adapter-finetuned.pth"
pretrained_path = "/content/checkpoints/lit-llama/7B/lit-llama.pth"
tokenizer_path = "/content/checkpoints/lit-llama/tokenizer.model"


fabric = L.Fabric(devices=1)
dtype = torch.bfloat16 if fabric.device.type == "cuda" and torch.cuda.is_bf16_supported() else torch.float32


The following function runs slowly, but sadly is needed to call for every generation ....

In [ ]:
def gen_title(pretrained_path, adapter_path, sample, max_new_tokens, top_k, temperature):

  """ Expects sample as set up above / like in the .pt data """

  with lazy_load(pretrained_path) as pretrained_checkpoint, lazy_load(adapter_path) as adapter_checkpoint:
      name = llama_model_lookup(pretrained_checkpoint)

      with EmptyInitOnDevice(
              device=fabric.device, dtype=dtype, quantization_mode = "llm.int8"
      ):
      #   quantization
          model = LLaMA.from_name(name)


      # 1. Load the pretrained weights
      model.load_state_dict(pretrained_checkpoint, strict=False)
      # 2. Load the fine-tuned adapter weights
      model.load_state_dict(adapter_checkpoint, strict=False)


  model.eval()
  model = fabric.setup_module(model)

  tokenizer = Tokenizer(tokenizer_path)

  prompt = generate_prompt(sample)
  encoded = tokenizer.encode(prompt, bos=True, eos=False, device=model.device)
  prompt_length = encoded.size(0)

  y = generate(model, encoded, max_new_tokens, temperature=temperature, top_k=top_k, eos_id=tokenizer.eos_id)

  output = tokenizer.decode(y)
  output = output.split("### Response:")[1].strip()

  return output


In [ ]:
## rouge with  jusst 1000 samples

test_set_ = test_set[:1000]


In [ ]:
## looping through eval/test samples

gen_titles_eval = []

for i in range(len(test_set_)):
  sample_ = test_set_[i]
 # print(i/len(test_set_))
  gen_title_new = gen_title(pretrained_path, adapter_path, sample_, 30, 200, 0.2)
  print(gen_title_new)
  gen_titles_eval.append(gen_title_new)



In [ ]:
titles_eval_true = [a["output"] for a in test_set_]

In [ ]:
processed_preds = postprocess_text(gen_titles_eval)
processed_labels = postprocess_text(titles_eval_true)


In [ ]:
!pip install evaluate
!pip install rouge_score

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
  Preparing metadata (setup.py) ... done
  Created wheel for rouge_score: filename=rouge_score-0.1.2-py3-none-any.whl size=24934 sha256=04abdd9c0f77f508cf2e6f19d16a4c88d1e60aa36f1f035eaed198708d528555
  Stored in directory: /root/.cache/pip/wheels/5f/dd/89/461065a73be61a532ff8599a28e9beef17985c9e9c31e541b4
Successfully built rouge_score


In [ ]:
#processed_preds_df = pd.read_csv("/content/gdrive/My Drive/Thesis/loss_data/preds_rouge.csv")
#processed_preds = processed_preds_df.preds



In [ ]:
import evaluate
rouge_score = evaluate.load("rouge")

rouge_score.add_batch(predictions=processed_preds, references=processed_labels)

result = rouge_score.compute()

In [ ]:
print("The ROUGE metrics on the evaluation data AFTER LORA-PEFT are:")
print(f'     F1 - ROUGE-1: 0.2526.')
print(f'     F1 - ROUGE-2: 0.0903.')
print(f'     F1 - ROUGE-L: 0.2419.')

The ROUGE metrics on the evaluation data AFTER LORA-PEFT are:
     F1 - ROUGE-1: 0.2526.
     F1 - ROUGE-2: 0.0903.
     F1 - ROUGE-L: 0.2419.


### Saving generated titles

Because this process is extremely slow (even with large A 100 GPU) I shall only generate titles for a subsample!

In [ ]:

sub_df = airbnb_london_filtered_images.sample(n = 1000)[["id", "description", "name"]]


In [ ]:
sub_df["gen_title"] = ""

In [ ]:
sub_df.index = list(range(sub_df.shape[0]))

In [ ]:
## setting up new dict

data_dict_sub_df = []

for i in range(sub_df.shape[0]):

  data_dict_sub_df.append({
      "instruction": "Summarize the following description into a short, appealing title for an AirBnB listing.",
      "input":   sub_df.description[i],
      "output": sub_df.name[i]
  })


max_seq_length = 256
mask_inputs = True
gen_set = [prepare_sample(sample, tokenizer, max_seq_length, mask_inputs) for sample in data_dict_sub_df]


In [ ]:
list_ids_ = list(sub_df.id)

In [ ]:

gen_titles = []
list_ids = []

for i in range(len(gen_set)):
  sample_ = gen_set[i]
  print(i/len(gen_set))
  gen_title_new = gen_title(pretrained_path, adapter_path, sample_, 30, 200, 0.2)
  gen_titles.append(gen_title_new)
  list_ids.append(list_ids_[i])

  if i % 100 == 0:
    sub_df_lora = pd.DataFrame({"id": list_ids, "gen_titles":gen_titles})
    sub_df_lora.to_csv("/content/gdrive/My Drive/Thesis/loss_data/gen_titles_llama_v2.csv", index = False)

sub_df_lora = pd.DataFrame({"id": list_ids, "gen_titles":gen_titles})
sub_df_lora.to_csv("/content/gdrive/My Drive/Thesis/loss_data/gen_titles_llama_v2.csv", index = False)


In [ ]:
 # since this code in running overnight in Colab and no resources shall be wasted:

from google.colab import runtime
runtime.unassign()